### NLP SemEval 2021 Task 1 - Lexical Complexity Prediction

In [ ]:
!pip install syllables

In [ ]:
# %%bash
# rm -rf sample_data

In [ ]:
import torch, os, time, io
import numpy as np
from tqdm import tqdm
from scipy import spatial 

In [ ]:
os.environ["WANDB_API_KEY"] = "4c52d1a3ccf7a8ea30016ee69d1e26ce2f9d0e67"

In [ ]:
data_drive_location = "/content/drive/MyDrive/Data"
print(os.getcwd())
if not os.getcwd() == data_drive_location:
  os.chdir(data_drive_location)

/content


In [ ]:
# # import SUBTLEX-US for word with freq>1
# %%bash
# wget -q 'https://www.ugent.be/pp/experimentele-psychologie/en/research/documents/subtlexus/subtlexus4.zip' -O 'SUBTLEXUS.zip'
# unzip -o 'SUBTLEXUS.zip'
# mv "SUBTLEXusfrequencyabove1.xls" "SUBTLEXUS.xls"

Count-based and surface form features

*   Length of word
*   Syllable Count
*   Word Frequency Using Subtlex-US*






In [ ]:
import syllables
import nltk, math
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
from collections import defaultdict
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
stemmer = PorterStemmer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def read_subtlex(filename="SUBTLEXUS.xls"):
  dfs = pd.read_excel(filename)
  return dfs

In [ ]:
def get_log_freq_from_SUBTLEX():
  frame = read_subtlex()
  frequency = defaultdict(float)
  for i in tqdm(range(len(frame))):
    word = str(frame.Word[i])
    SUBTLwf = frame.SUBTLWF[i]
    token = stemmer.stem(word.lower())
    frequency[token] += float(SUBTLwf)
  log_freq = {k: math.log2(v) for k,v in frequency.items()}
  return log_freq

In [ ]:
log_freq = get_log_freq_from_SUBTLEX()
def surface_features(word):
  freq = 0
  try:
    freq = log_freq[word]
  except:
    pass

  num_upper = len([_ for _ in word if _.isupper()])/len(word)
  return [len(word), syllables.estimate(word), freq, num_upper]

100%|██████████| 60384/60384 [00:03<00:00, 19308.59it/s]


In [ ]:
# # POS DICTIONARY
# # To be used for embeddings
# posdict = {"$", "''","(",")",",","--",".", ":", "CC", "CD", "DT", "EX", "FW", "IN","JJ","JJR","JJS","LS","MD","NN","NNP","NNPS", "NNS","PDT","POS","PRP","PRP$","RB","RBR","RBS", "RP","SYM", "TO", "UH","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP","WP$","WRB","``"}
# posdict = {y:x for x,y in enumerate(posdict)}
# print(posdict)

Morphological Features
*   Part-of-speech
*   Length of suffix from Stemmed Word



In [ ]:
def morphological_features(word):
  word = word.lower()
  # pos^, stemmer (len(word) - len(stem))
  #    ^ might need to omit
  # pos = nltk.pos_tag([word])[0][1]
    # print(pos, posdict[pos])
  word_stemmed = stemmer.stem(word)
  # return [posdict[pos], len(word) - len(word_stemmed)]
  return [len(word) - len(word_stemmed)]

Semantic Features
*   Degree of Hypernyms
* Degree of Hyponyms
*   Degree of Polysemy



In [ ]:
def semantic_features(word):
  # polysemy and homonymy* 
  #                      *not doing for now
  try:
    results = wordnet.synsets(word)
  except:
    return [0,0,0]
  polysemy = len(results)
  hypernyms=0
  hyponyms=0
  try:
      hypernyms = len(results[0].hypernyms())
      hyponyms = len(results[0].hyponyms())
  except:
    pass
  return [polysemy, hypernyms, hyponyms]

Singularity/Plurality

In [ ]:
#Plurality Singularity Feature using inflect
import sys
sys.path.insert(0, "./src/inflect")
import inflect
def Singularity_Plurality(word):
  engine = inflect.engine()
  is_Singular=False
  is_Plural=False
  is_singular_plural=False
  if(engine.singular_noun(word)==False):
    is_Singular=True
  else:
    is_Plural=True
  if(engine.plural(word)==word):
    is_singular_plural=True
  if(is_singular_plural):
    Result=[0,0,1]
  else:
    if(is_Singular):
      Result=[1,0,0]
    else:
      Result=[0,1,0]
  return Result

In [ ]:
ogden_dictionary={}
with open("ogden.txt","r") as f:
  for line in f.readlines():
    ogden_dictionary[line.strip()]=1
print(len(ogden_dictionary))
def ogden_word_set(word):
  if(word in ogden_dictionary.keys()):
    return [1]
  else:
    return [0]

848


In [ ]:
wiki_dictionary={}
with open("wiki.txt","r") as f:
  for line in f.readlines():
    wiki_dictionary[line.strip()]=1
print(len(wiki_dictionary))
def wiki_word_set(word):
  if(word in wiki_dictionary.keys()):
    return [1]
  else:
    return [0]

2019


In [ ]:
subIMDB_dictionary={}
import re
with open("subtitles.txt","r") as f:
  for line in f.readlines():
    res = re.sub(r'[^\w\s]', '',line)
    for x in res.split():
      subIMDB_dictionary[x]=1
def subIMDB_word_set(word):
  if(word in subIMDB_dictionary.keys()):
    return [1]
  else:
    return [0]

In [ ]:
# !pip install python-datamuse
# from datamuse import datamuse
# api = datamuse.Datamuse()
# def google_word_frequency(word):
#   try:
#     result=api.words(sp=word, max=1, md='f')
#     return [float(result[0]["tags"][0][2:])]
#   except:
#     return [0.000]

In [ ]:
def bindal_features(word):
  word=word.lower()
  word=res = re.sub(r'[^\w\s]', '',word)
  return Singularity_Plurality(word)+ogden_word_set(word)+wiki_word_set(word)+subIMDB_word_set(word)

In [ ]:
def handcrafted(word):
  return surface_features(word) + morphological_features(word) + semantic_features(word)+bindal_features(word)

In [ ]:
def handcrafted_multi(multiword):
  first, second = multiword.split()
  avg = [(x+y)/2 for x,y in zip(surface_features(first)+morphological_features(first),surface_features(second)+morphological_features(second))]
  second_features = semantic_features(second)+bindal_features(second)

  return avg + second_features


In [ ]:
handcrafted_multi("fast racer")

[4.5, 1.5, 4.660190562456907, 0.0, 0.0, 0, 0, 0, 1, 0, 0, 0, 0, 1]

In [ ]:
handcrafted("Racer")

[5, 2, 0, 0.2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]

DOWNLOAD DATASET

In [ ]:
# # DOWNLOAD DATASET
# %%bash
# mkdir dataset
# cd dataset
# mkdir train
# wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv' -O 'train/lcp_single_train.tsv'
# wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_multi_train.tsv' -O 'train/lcp_multi_train.tsv'

# mkdir trial
# wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/trial/lcp_single_trial.tsv' -O 'trial/lcp_single_trial.tsv'
# wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/trial/lcp_multi_trial.tsv' -O 'trial/lcp_multi_trial.tsv'

# mkdir test
# wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test/lcp_single_test.tsv' -O 'test/lcp_single_test.tsv'
# wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test/lcp_multi_test.tsv' -O 'test/lcp_multi_test.tsv'

#GloVe

Using pre-trained glove to perform some embeddings


 


In [ ]:
# %%bash
# wget -p "http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip" -O "glove.840B.300d.zip"
# unzip "glove.840B.300d.zip"

In [ ]:
# %%bash
# wget -p "https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip" -O "glove.42B.300d.zip"
# unzip "glove.42B.300d.zip"

In [ ]:
# %%bash
# wget -p "http://nlp.stanford.edu/data/glove.6B.zip" -O "glove.6B.zip"
# unzip "glove.6B.zip"

In [ ]:
def get_glove_w2v(filename):
  with open(filename,"r",encoding="utf8") as lines:
    glove_w2v = {}
    # count=0
    for line in tqdm(lines):
      # count+=1
      # if(count>10):
        # break
      values = line.split()
      # print(line[:-300],"\t%%%%%%%%\t", line[-300:])
      word = ''.join(values[:-300])
      vector = np.asarray(values[-300:],dtype='float32')
      glove_w2v[word.lower()] = vector
    print("\nEmbeddings loaded from glove:",len(glove_w2v))
    return glove_w2v

In [ ]:
glove_w2v_loc = "glove.6B.300d.txt"
glove_w2v = get_glove_w2v(glove_w2v_loc)
# glove_w2v_
# glove_w2v_large_loc = "glove.840B.300d.txt"
# glove_w2v_large = get_glove_w2v(glove_w2v_large_loc)

400000it [00:41, 9653.09it/s] 


Embeddings loaded from glove: 400000


In [ ]:
# %%bash
# mkdir fastText
# curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
# unzip fastText/crawl-300d-2M.vec.zip -d fastText/

In [ ]:
# %%bash
# mkdir encoder
# curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

In [ ]:
# import io

# def load_vectors(fname):
#     fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
#     n, d = map(int, fin.readline().split())
#     data = {}
#     for line in fin:
#         tokens = line.rstrip().split(' ')
#         data[tokens[0]] = map(float, tokens[1:])
#     return data

In [ ]:
# from models import InferSent
# V = 2
# MODEL_PATH = 'encoder/infersent%s.pkl' % V
# params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
#                 'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
# infersent = InferSent(params_model)
# infersent.load_state_dict(torch.load(MODEL_PATH))

In [ ]:
# W2V_PATH = 'fastText/crawl-300d-2M.vec'
# infersent.set_w2v_path(W2V_PATH)

In [ ]:
# fasttext_w2v = load_vectors(W2V_PATH)

In [ ]:
#infersent.build_vocab_k_words(K=5000000)

# Sentence Bert

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer


We will use "stsb-distilbert-base" as our model for the task.
It provides an performance (Pearson Scale) > 80% on STSb dataset and has a speed of 4000 sentences/ sec on V1000GPU, which is the highest among the pretrained models available at SBert.net: https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0

We will also try out Roberta which has even higher Performance at the cost of speed

In [ ]:
sbert_model = SentenceTransformer('stsb-distilbert-base')
# sbert_model = SentenceTransformer('stsb-roberta-large')

In [ ]:
# sbert_model.eval()

In [ ]:
# roberta_model = SentenceTransformer('stsb-roberta-base')

In [ ]:
def get_embedding_for_context(ctx, model):
  if not isinstance(ctx, list):
    ctx = [ctx]
  return model.encode(ctx)

In [ ]:
# def test_models(model):
#   start = time.time()
#   get_embedding_for_context("This is test sentence", model)
#   print("Time for single prediction: ", time.time()-start)
#   start = time.time()
#   get_embedding_for_context(["This is test sentence"]*3000, model)
#   print("Time for 3,000 prediction: ", time.time()-start)

In [ ]:
# test_models(sbert_model)
# test_models(infersent)

In [ ]:
# test_models(roberta_model)

# Data Collection

In [ ]:
import csv
def get_data_print_details(filepath, crps = 'corpus'):
  df_path_single = pd.read_csv(filepath, quoting=csv.QUOTE_NONE, sep='\t', header=0)
  print("Data columns:")
  print(df_path_single.columns)
  print("\nTotal corpus len: {}".format(len(df_path_single)))
  print("\nSubcorpus len:")
  print(df_path_single[crps].value_counts())
  return df_path_single

#sINGLE

Train Data

In [ ]:
train_single_tsv = 'dataset/train/lcp_single_train.tsv'

In [ ]:
df_train_single = get_data_print_details(train_single_tsv)

Data columns:
Index(['id', 'corpus', 'sentence', 'token', 'complexity'], dtype='object')

Total corpus len: 7662

Subcorpus len:
biomed      2576
bible       2574
europarl    2512
Name: corpus, dtype: int64


Test Data

In [ ]:
test_single_tsv = 'dataset/test/lcp_single_test.tsv'
df_test_single = get_data_print_details(test_single_tsv)

Data columns:
Index(['id', 'corpus', 'sentence', 'token'], dtype='object')

Total corpus len: 917

Subcorpus len:
europarl    345
biomed      289
bible       283
Name: corpus, dtype: int64


Trail Data

In [ ]:
trial_single_tsv = 'dataset/trial/lcp_single_trial.tsv'
df_trial_single = get_data_print_details(trial_single_tsv,'subcorpus')

Data columns:
Index(['id', 'subcorpus', 'sentence', 'token', 'complexity'], dtype='object')

Total corpus len: 421

Subcorpus len:
bible       143
europarl    143
biomed      135
Name: subcorpus, dtype: int64


#mULTI

In [ ]:
train_multi_tsv = 'dataset/train/lcp_multi_train.tsv'
df_train_multi = get_data_print_details(train_multi_tsv)

Data columns:
Index(['id', 'corpus', 'sentence', 'token', 'complexity'], dtype='object')

Total corpus len: 1517

Subcorpus len:
biomed      514
bible       505
europarl    498
Name: corpus, dtype: int64


In [ ]:
test_multi_tsv = 'dataset/test/lcp_multi_test.tsv'
df_test_multi = get_data_print_details(test_multi_tsv)

Data columns:
Index(['id', 'corpus', 'sentence', 'token'], dtype='object')

Total corpus len: 184

Subcorpus len:
bible       66
europarl    65
biomed      53
Name: corpus, dtype: int64


In [ ]:
trial_multi_tsv = 'dataset/trial/lcp_multi_trial.tsv'
df_trial_multi = get_data_print_details(trial_multi_tsv,'subcorpus')

Data columns:
Index(['id', 'subcorpus', 'sentence', 'token', 'complexity'], dtype='object')

Total corpus len: 99

Subcorpus len:
europarl    37
biomed      33
bible       29
Name: subcorpus, dtype: int64


#LOAD DATASET

Load DataSet

In [ ]:
from torch.utils.data import Dataset

In [ ]:
def preprocess_embeddings(data, model):
  sent_embed = {}
  all_sent = data['sentence'].tolist()

  start= time.time()
  all_sent_embed = get_embedding_for_context(all_sent, model)
  print("All sentences Embedded in :", time.time()-start)
  return all_sent_embed

In [ ]:
# embed_sbert = preprocess_embeddings(df_trial_single, sbert_model)
# print(embed_sbert.shape, embed_sbert)

In [ ]:
# embed_roberta = preprocess_embeddings(df_trial_single, roberta_model)
# print(embed_roberta.shape, embed_roberta)

In [ ]:
class CompLexDataset(Dataset):
  # global dataset_type

  def __init__ (self, d_type, df_type_single, model):
    self.dataset_type = d_type
    self.df_single = df_type_single
    self.all_sentence_embeddings = preprocess_embeddings(df_type_single, model)

  def __len__ (self):
    return len(self.df_single)
  # len(datasetobj) == datasetobj.__len__()
  
  # datasetobj[5] == datasetobj.__getitem__(5)
  def __getitem__ (self, idx):
    token = str(self.df_single.loc[idx, 'token'])
    try:
      complexity = self.df_single.loc[idx, 'complexity']
    except:
      complexity = float("NaN")
    handcrafted_features = handcrafted(token)
    sentence_context = self.all_sentence_embeddings[idx]
    token_w2v = (glove_w2v.get(token.lower(), [0]*300))

    result = {
        'in' : (torch.from_numpy(np.hstack((np.array(handcrafted_features), sentence_context, np.array(token_w2v))).ravel()).float()),
        'out': torch.from_numpy(np.array([complexity])).float()
    }

    return result

In [ ]:
class CompLexDatasetMulti(Dataset):
  # global dataset_type

  def __init__ (self, d_type, df_type_multi, model):
    self.dataset_type = d_type
    self.df_multi = df_type_multi
    self.all_sentence_embeddings = preprocess_embeddings(df_type_multi, model)

  def __len__ (self):
    return len(self.df_multi )
  # len(datasetobj) == datasetobj.__len__()
  
  # datasetobj[5] == datasetobj.__getitem__(5)
  def __getitem__ (self, idx):
    token = str(self.df_multi.loc[idx, 'token'])
    try:
      complexity = self.df_multi.loc[idx, 'complexity']
    except:
      complexity = float("NaN")
    handcrafted_features = handcrafted_multi(token)
    sentence_context = self.all_sentence_embeddings[idx]
    first, second = token.split()
    first_w2v = (glove_w2v.get(first.lower(), [0]*300))
    second_w2v = (glove_w2v.get(second.lower(), [0]*300))
    result = {
        'in' : (torch.from_numpy(np.hstack((np.array(handcrafted_features), sentence_context, np.array(first_w2v),np.array(second_w2v))).ravel()).float()),
        'out': torch.from_numpy(np.array([complexity])).float()
    }

    return result

In [ ]:
def print_details(a):
  pass
  # print("Input: ", a[5]['in'], "Input Length: ", len(a[5]['in']))
  # print("Output: ", a[5]['out'])

#Single Dataset

Train

In [ ]:
train_dataset_single = CompLexDataset("train", df_train_single, sbert_model)
print_details(train_dataset_single)

All sentences Embedded in : 585.9065644741058


Trial

In [ ]:
trial_dataset_single = CompLexDataset("trial", df_trial_single, sbert_model)
print_details(trial_dataset_single)

All sentences Embedded in : 35.28865838050842


Test

In [ ]:
test_dataset_single = CompLexDataset("test", df_test_single, sbert_model)
print_details(test_dataset_single)

All sentences Embedded in : 72.02329659461975


#Multi Dataset

In [ ]:
train_dataset_multi = CompLexDatasetMulti("train", df_train_multi, sbert_model)
print_details(train_dataset_multi)

All sentences Embedded in : 119.7036304473877


In [ ]:
trial_dataset_multi = CompLexDatasetMulti("trial", df_trial_multi, sbert_model)
print_details(trial_dataset_multi)

All sentences Embedded in : 9.501445055007935


In [ ]:
test_dataset_multi = CompLexDatasetMulti("test", df_test_multi, sbert_model)
print_details(test_dataset_multi)

All sentences Embedded in : 15.824579238891602


# Neural Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dim_hidden = 1600
class Network_base(nn.Module):
  
  def __init__ (self, inp_layer):
    super().__init__()

    self.fc1 = nn.Linear(inp_layer, dim_hidden)
    self.fc2 = nn.Linear(dim_hidden, dim_hidden)
    self.fc3 = nn.Linear(dim_hidden,1)

    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    return self.sigmoid(self.fc3(self.fc2(self.fc1(x))))

In [ ]:
# dim_hidden = 800
# class Network_base_sodhi(nn.Module):
  
#   def __init__ (self, inp_layer):
#     super().__init__()

#     self.fc1 = nn.Linear(inp_layer, dim_hidden)
#     self.fc2 = nn.Linear(dim_hidden, dim_hidden)
#     self.fc3 = nn.Linear(dim_hidden, dim_hidden)
#     self.fc4 = nn.Linear(dim_hidden, dim_hidden)
#     self.fc5 = nn.Linear(dim_hidden,1)

#     self.sigmoid = nn.Sigmoid()

#   def forward(self, x):
#     return self.sigmoid(self.fc5(self.fc4(self.fc3(self.fc2(self.fc1(x))))))

In [ ]:
network_single = Network_base(len(train_dataset_single[0]['in']))

In [ ]:
network_multi  = Network_base(len(train_dataset_multi[0]['in']))

TRAIN

In [ ]:
!pip install wandb -q
import wandb
wandb.login()

     |████████████████████████████████| 2.1MB 5.0MB/s 
     |████████████████████████████████| 133kB 34.9MB/s 
     |████████████████████████████████| 102kB 8.9MB/s 
     |████████████████████████████████| 163kB 25.4MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 


wandb: Currently logged in as: nlpsangh (use `wandb login --relogin` to force relogin)


True

In [ ]:
from torch.optim import Adam
from torch.optim import AdamW
from torch.utils.data import DataLoader

# Single

In [ ]:
network_in_use = network_single

In [ ]:
wandb.init()
config = wandb.config
config.learning_rate = 1e-6
config.batch_size=64
config.num_steps=500
config.display_step=5
wandb.watch(network_in_use)

In [ ]:
#HYPER-PARAMS
criterion = nn.MSELoss()
optm = AdamW(network_in_use.parameters(), config.learning_rate)

In [ ]:
def trainpass(net, x, y, optimizer, criterion):
  net.zero_grad()
  output = net(x)
  loss = criterion(output,y)
  loss.backward()
  optimizer.step()

  return loss, output

In [ ]:
data_train = DataLoader(dataset = train_dataset_single, batch_size = config.batch_size, shuffle= True)

In [ ]:
def save_state(network, optm, epoch, filename):
  state_dict = {'model_state_dict': network.state_dict(),
                  'optimizer_state_dict': optm.state_dict(),
                  'epochs': epoch}
  torch.save(state_dict,filename)

In [ ]:
for step in range(1, config.num_steps + 1):
  epoch_loss=0  
  for idx, batch in enumerate(data_train):
      x_train = batch['in']
      y_train = batch['out']
      loss, pred =  trainpass(network_in_use, x_train, y_train, optm, criterion)
      epoch_loss += loss

  if step % config.display_step == 0 or step == 1:
    wandb.log({"loss":epoch_loss})

  # try:
  #   if(epoch_loss < best_loss):
  #     best_loss = epoch_loss
  #     save_state(network2, optm, epoch, '2803_sbert_network_'+str(int(time.time()%100000))+'.pt')
  #     save_state(network2, optm, epoch, '2803_sbert_network_best.pt')
  # except:
  #   best_loss= epoch_loss

using Sbert

In [ ]:
state_dict = {'model_state_dict': network_in_use.state_dict(),
                  'optimizer_state_dict': optm.state_dict(),
              }
torch.save(state_dict,"sbert_network-evalution-single.pt")

In [ ]:
# def load_best_state(network, optm,filename):
#   state_dict = torch.load(filename)
#   network.load_state_dict(state_dict['model_state_dict'])
#   optm.load_state_dict(state_dict['optimizer_state_dict'])

In [ ]:
# load_best_state(network_in_use, optm,"sbert_network-evalution-single.pt")

Validating

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import pearsonr as pnr
import csv

Train

In [ ]:
def test_train(dataset, df,network, flag=False, filename='', test=False):
  y_true = [dataset[i]['out'].item() for i in range(len(dataset))]
  y_pred = []

  test_loader = DataLoader(dataset=dataset, batch_size=config.batch_size, shuffle=False)

  for id, batch in enumerate(test_loader):
    x_train = batch['in']
    y_pred.append(network(x_train))

  y_pred = [x.item() for i in range(len(y_pred)) for x in y_pred[i]]
  print(y_pred)
  if test==False:
    val_mae = mae(y_true, y_pred)
    val_pearsonr = pnr(y_true, y_pred)
    print("MAE: ",val_mae, "\tPearsonr: ", val_pearsonr)
  
  if flag==True:
    with open(filename, 'w', newline='') as f:
      f_writer = csv.writer(f, delimiter=',')
      for id in range(len(df)):
        f_writer.writerow((df.loc[id, 'id'], str(y_pred[id])))

In [ ]:
test_train(train_dataset_single, df_train_single, network_in_use)

[0.22415991127490997, 0.17541684210300446, 0.2125495970249176, 0.22355882823467255, 0.21435058116912842, 0.1975022554397583, 0.21471944451332092, 0.178202822804451, 0.15063026547431946, 0.1618044674396515, 0.2224280834197998, 0.2183844894170761, 0.2771015763282776, 0.25990769267082214, 0.26133134961128235, 0.2798393666744232, 0.3284761309623718, 0.19012141227722168, 0.33196502923965454, 0.30116644501686096, 0.3095912039279938, 0.28149986267089844, 0.18960216641426086, 0.25684061646461487, 0.20215730369091034, 0.2643532454967499, 0.22487279772758484, 0.31317251920700073, 0.18874207139015198, 0.2534319758415222, 0.2331533581018448, 0.21823380887508392, 0.23648998141288757, 0.3614262640476227, 0.3429392874240875, 0.35531970858573914, 0.2882936894893646, 0.3004584014415741, 0.31260260939598083, 0.23570814728736877, 0.22453653812408447, 0.2385241538286209, 0.24619458615779877, 0.3162193298339844, 0.23620855808258057, 0.23134823143482208, 0.21139004826545715, 0.20464280247688293, 0.231445267

In [ ]:
test_train(trial_dataset_single, df_trial_single, network_in_use, True, 'trial-result-evaluation-single.csv')

[0.2931492328643799, 0.3160709738731384, 0.37945669889450073, 0.36522045731544495, 0.3316907286643982, 0.3525654375553131, 0.3631291687488556, 0.3752806782722473, 0.38426443934440613, 0.2858268618583679, 0.3318265974521637, 0.27454060316085815, 0.26327642798423767, 0.3229014575481415, 0.48698946833610535, 0.5461401343345642, 0.26267513632774353, 0.45259982347488403, 0.31384873390197754, 0.37336307764053345, 0.41976940631866455, 0.5159792900085449, 0.46393242478370667, 0.4303332269191742, 0.4802076816558838, 0.31900322437286377, 0.4161975383758545, 0.5421962738037109, 0.4049365818500519, 0.3053271472454071, 0.3246041238307953, 0.3200388550758362, 0.27476266026496887, 0.27594366669654846, 0.3339114785194397, 0.3124227523803711, 0.37575021386146545, 0.3052084445953369, 0.5288273096084595, 0.48134085536003113, 0.5247827172279358, 0.5364285111427307, 0.46789032220840454, 0.5218992233276367, 0.6726301312446594, 0.6055821180343628, 0.4048183858394623, 0.592890739440918, 0.6517005562782288, 0.

In [ ]:
test_train(test_dataset_single,df_test_single, network_in_use, True, 'test-result-evaluation-single.csv', True)

[0.2992874085903168, 0.3472228944301605, 0.3519381582736969, 0.3060207664966583, 0.49332043528556824, 0.291726291179657, 0.39221397042274475, 0.3044189214706421, 0.3645820915699005, 0.2178807407617569, 0.5249123573303223, 0.4272586405277252, 0.46233898401260376, 0.261376291513443, 0.5991198420524597, 0.509066641330719, 0.4740954637527466, 0.26640793681144714, 0.43963709473609924, 0.4569171071052551, 0.5153912901878357, 0.4854099452495575, 0.2835313379764557, 0.4391140341758728, 0.41370850801467896, 0.5049676895141602, 0.2780631482601166, 0.3048694133758545, 0.29248663783073425, 0.4535919427871704, 0.22636422514915466, 0.5581715106964111, 0.5425606369972229, 0.4655139446258545, 0.5397340655326843, 0.38635048270225525, 0.4962138235569, 0.5354791283607483, 0.27469050884246826, 0.253020316362381, 0.5565029978752136, 0.542026937007904, 0.47623002529144287, 0.3044593334197998, 0.28551986813545227, 0.30164116621017456, 0.13225075602531433, 0.2820371091365814, 0.29598015546798706, 0.4100893437

# Multi

In [ ]:
network_in_use = network_multi

In [ ]:
wandb.init()
config = wandb.config
config.learning_rate = 4e-7
config.batch_size=32
config.num_steps=350
config.display_step=5
wandb.watch(network_in_use)

In [ ]:
#HYPER-PARAMS
criterion = nn.MSELoss()
optm = AdamW(network_in_use.parameters(), config.learning_rate)

In [ ]:
def trainpass(net, x, y, optimizer, criterion):
  net.zero_grad()
  output = net(x)
  loss = criterion(output,y)
  loss.backward()
  optimizer.step()

  return loss, output

In [ ]:
data_train = DataLoader(dataset = train_dataset_single, batch_size = config.batch_size, shuffle= True)

In [ ]:
def save_state(network, optm, epoch, filename):
  state_dict = {'model_state_dict': network.state_dict(),
                  'optimizer_state_dict': optm.state_dict(),
                  'epochs': epoch}
  torch.save(state_dict,filename)

In [ ]:
for step in range(1, config.num_steps + 1):
  epoch_loss=0  
  for idx, batch in enumerate(data_train):
      x_train = batch['in']
      y_train = batch['out']
      loss, pred =  trainpass(network_in_use, x_train, y_train, optm, criterion)
      epoch_loss += loss

  if step % config.display_step == 0 or step == 1:
    wandb.log({"loss":epoch_loss})

  # try:
  #   if(epoch_loss < best_loss):
  #     best_loss = epoch_loss
  #     save_state(network2, optm, epoch, '2803_sbert_network_'+str(int(time.time()%100000))+'.pt')
  #     save_state(network2, optm, epoch, '2803_sbert_network_best.pt')
  # except:
  #   best_loss= epoch_loss

using Sbert

In [ ]:
state_dict = {'model_state_dict': network_in_use.state_dict(),
                  'optimizer_state_dict': optm.state_dict(),
              }
torch.save(state_dict,"sbert_network-evaluation-multi.pt")

In [ ]:
# def load_best_state(network, optm):
#   state_dict = torch.load('sbert_network_25_best.pt')
#   network.load_state_dict(state_dict['model_state_dict'])
#   optm.load_state_dict(state_dict['optimizer_state_dict'])

In [ ]:
# load_best_state(network_in_use, optm)

Validating

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import pearsonr as pnr
import csv

Train

In [ ]:
def test_train(dataset, df,network, flag=False, filename='', test=False):
  y_true = [dataset[i]['out'].item() for i in range(len(dataset))]
  y_pred = []

  test_loader = DataLoader(dataset=dataset, batch_size=config.batch_size, shuffle=False)

  for id, batch in enumerate(test_loader):
    x_train = batch['in']
    y_pred.append(network(x_train))

  y_pred = [x.item() for i in range(len(y_pred)) for x in y_pred[i]]
  print(y_pred)
  if test==False:
    val_mae = mae(y_true, y_pred)
    val_pearsonr = pnr(y_true, y_pred)
    print("MAE: ",val_mae, "\tPearsonr: ", val_pearsonr)
  
  if flag==True:
    with open(filename, 'w', newline='') as f:
      f_writer = csv.writer(f, delimiter=',')
      for id in range(len(df)):
        f_writer.writerow((df.loc[id, 'id'], str(y_pred[id])))

In [ ]:
test_train(train_dataset_single, df_train_single, network_in_use)

[0.22415991127490997, 0.17541684210300446, 0.2125495970249176, 0.22355882823467255, 0.21435058116912842, 0.1975022554397583, 0.21471944451332092, 0.178202822804451, 0.15063026547431946, 0.1618044674396515, 0.2224280834197998, 0.2183844894170761, 0.2771015763282776, 0.25990769267082214, 0.26133134961128235, 0.2798393666744232, 0.3284761309623718, 0.19012141227722168, 0.33196502923965454, 0.30116644501686096, 0.3095912039279938, 0.28149986267089844, 0.18960216641426086, 0.25684061646461487, 0.20215730369091034, 0.2643532454967499, 0.22487279772758484, 0.31317251920700073, 0.18874207139015198, 0.2534319758415222, 0.2331533581018448, 0.21823380887508392, 0.23648998141288757, 0.3614262640476227, 0.3429392874240875, 0.35531970858573914, 0.2882936894893646, 0.3004584014415741, 0.31260260939598083, 0.23570814728736877, 0.22453653812408447, 0.2385241538286209, 0.24619458615779877, 0.3162193298339844, 0.23620855808258057, 0.23134823143482208, 0.21139004826545715, 0.20464280247688293, 0.231445267

In [ ]:
test_train(trial_dataset_single, df_trial_single, network_in_use, True, 'trial-result-evaluation-multi.csv')

[0.2931492328643799, 0.3160709738731384, 0.37945669889450073, 0.36522045731544495, 0.3316907286643982, 0.3525654375553131, 0.3631291687488556, 0.3752806782722473, 0.38426443934440613, 0.2858268618583679, 0.3318265974521637, 0.27454060316085815, 0.26327642798423767, 0.3229014575481415, 0.48698946833610535, 0.5461401343345642, 0.26267513632774353, 0.45259982347488403, 0.31384873390197754, 0.37336307764053345, 0.41976940631866455, 0.5159792900085449, 0.46393242478370667, 0.4303332269191742, 0.4802076816558838, 0.31900322437286377, 0.4161975383758545, 0.5421962738037109, 0.4049365818500519, 0.3053271472454071, 0.3246041238307953, 0.3200388550758362, 0.27476266026496887, 0.27594366669654846, 0.3339114785194397, 0.3124227523803711, 0.37575021386146545, 0.3052084445953369, 0.5288273096084595, 0.48134085536003113, 0.5247827172279358, 0.5364285111427307, 0.46789032220840454, 0.5218992233276367, 0.6726301312446594, 0.6055821180343628, 0.4048183858394623, 0.592890739440918, 0.6517005562782288, 0.

In [ ]:
test_train(test_dataset_single,df_test_single, network_in_use, True, 'test-result-evaluation-multi.csv', True)

[0.2992874085903168, 0.3472228944301605, 0.3519381582736969, 0.3060207664966583, 0.49332043528556824, 0.291726291179657, 0.39221397042274475, 0.3044189214706421, 0.3645820915699005, 0.2178807407617569, 0.5249123573303223, 0.4272586405277252, 0.46233898401260376, 0.261376291513443, 0.5991198420524597, 0.509066641330719, 0.4740954637527466, 0.26640793681144714, 0.43963709473609924, 0.4569171071052551, 0.5153912901878357, 0.4854099452495575, 0.2835313379764557, 0.4391140341758728, 0.41370850801467896, 0.5049676895141602, 0.2780631482601166, 0.3048694133758545, 0.29248663783073425, 0.4535919427871704, 0.22636422514915466, 0.5581715106964111, 0.5425606369972229, 0.4655139446258545, 0.5397340655326843, 0.38635048270225525, 0.4962138235569, 0.5354791283607483, 0.27469050884246826, 0.253020316362381, 0.5565029978752136, 0.542026937007904, 0.47623002529144287, 0.3044593334197998, 0.28551986813545227, 0.30164116621017456, 0.13225075602531433, 0.2820371091365814, 0.29598015546798706, 0.4100893437